### [VGG16 implementation using pytorch](https://blog.paperspace.com/vgg-from-scratch-pytorch/)

#### Import Libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
print(torch.__version__)

2.1.0+cu121


#### Device configuration

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#### Loading CIFAR data

In [4]:
def data_loader(data_dir,
                batch_size,
                random_seed=42,
                valid_size=0.1,
                shuffle=True,
                test=False):

    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    transform = transforms.Compose([
            transforms.Resize((227,227)),
            transforms.ToTensor(),
            normalize,
    ])

    if test:
        dataset = datasets.CIFAR100(
          root=data_dir, train=False,
          download=True, transform=transform,
        )

        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=shuffle
        )

        return data_loader

    # load the dataset
    train_dataset = datasets.CIFAR100(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)



In [5]:

# CIFAR100 dataset
train_loader, valid_loader = data_loader(data_dir='./data',
                                         batch_size=64)

test_loader = data_loader(data_dir='./data',
                              batch_size=64,
                              test=True)

100%|██████████| 169001437/169001437 [00:03<00:00, 47387734.16it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


100%|██████████| 170498071/170498071 [00:03<00:00, 48897713.08it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


#### Build VGG16 Model

In [6]:
class VGG16(nn.Module):
  def __init__(self, num_classes=10):
    super(VGG16, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3,out_channels=64,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(num_features=64),
        nn.ReLU()
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(num_features=64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.conv3 = nn.Sequential(
        nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU()
    )
    self.conv4 = nn.Sequential(
        nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)

    )
    self.conv5 = nn.Sequential(
        nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU()
    )
    self.conv6 = nn.Sequential(
        nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU()
    )
    self.conv7 = nn.Sequential(
        nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.conv8 = nn.Sequential(
        nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )
    self.conv9 = nn.Sequential(
        nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )
    self.conv10 = nn.Sequential(
        nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.conv11 = nn.Sequential(
        nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )
    self.conv12 = nn.Sequential(
        nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )
    self.conv13 = nn.Sequential(
        nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc1 = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(in_features=7*7*512, out_features=4096),
        nn.ReLU())
    self.fc2 = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(in_features=4096, out_features=4096),
        nn.ReLU())
    self.fc3 = nn.Sequential(
        nn.Linear(in_features=4096, out_features=num_classes))




  def forward(self, x):
    out = self.conv1(x)
    out = self.conv2(out)
    out = self.conv3(out)
    out = self.conv4(out)
    out = self.conv5(out)
    out = self.conv6(out)
    out = self.conv7(out)
    out = self.conv8(out)
    out = self.conv9(out)
    out = self.conv10(out)
    out = self.conv11(out)
    out = self.conv12(out)
    out = self.conv13(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc1(out)
    out = self.fc2(out)
    out = self.fc3(out)
    return out

#### Model Summary

In [9]:
# !pip install torchsummary
# from torchsummary import summary

In [10]:
# summary(model=VGG16(num_classes=10),
#         input_size=(3, 224,224))

In [11]:
!pip install torchinfo

In [12]:
import torchinfo
torchinfo.summary(VGG16(),input_size=(3,224,224),batch_dim=0, verbose = 0)

Layer (type:depth-idx)                   Output Shape              Param #
VGG16                                    [1, 10]                   --
├─Sequential: 1-1                        [1, 64, 224, 224]         --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─BatchNorm2d: 2-2                  [1, 64, 224, 224]         128
│    └─ReLU: 2-3                         [1, 64, 224, 224]         --
├─Sequential: 1-2                        [1, 64, 112, 112]         --
│    └─Conv2d: 2-4                       [1, 64, 224, 224]         36,928
│    └─BatchNorm2d: 2-5                  [1, 64, 224, 224]         128
│    └─ReLU: 2-6                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-7                    [1, 64, 112, 112]         --
├─Sequential: 1-3                        [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        73,856
│    └─BatchNorm2d: 2-9                  [1, 128, 112, 112]        256
│

#### Hyperparameters

In [13]:
num_classes = 100
num_epochs = 5
batch_size = 16
learning_rate = 0.001

# load model to device
model = VGG16(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)


# Train the model
total_step = len(train_loader)

#### Model Training

In [14]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))

KeyboardInterrupt: 

#### Testing

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))

